In [3]:
import numpy as np
import pandas as pd
import pycountry

### TRANSFORMACIONES ARCHIVOS MIGRACION NETA E INDICADORES BANCO MUNDIAL

1. Creo una lista con los nombres de los archivos y lo los nombres de los dataframes que los van a contener

In [82]:
lista_archivos=['Banco_Mundia_ Ingresos_Per_Capita_segun_encuestas.xls','Banco_Mundial_Acceso_A_Electricidad.xls','Banco_Mundial_Contaminacion_Aire.xls','Banco_Mundial_Crecimiento_del_PBI_%anual.xls','Banco_Mundial_Crecimiento_del_PBI_per_capita.xls','Banco_Mundial_Desempleo_Hombres.xls','Banco_Mundial_Desempleo_mujeres.xls','Banco_Mundial_Gasto_%_PBI.xls','Banco_Mundial_Importaciones_de_Mercaderia.xls','Banco_Mundial_Migration_Rate_Argentina.xls','Banco_Mundial_Personas_Desempleadas_Con_Educacion_Avanzada.xls','Banco_Mundial_Tasa_De_Cambio_Oficial.xls','Banco_Mundial_Tasa_De_Incidencia_De_La_Pobreza.xls','Banco_Mundial_Tasa_de_Mortalidad_en_un_año.xls','Banco_Mundial_Volumenes_internacionales_de_migrantes.xls']
lista_nombres_df=['i_ingresos','i_electricidad','i_contaminacion','i_crecimiento_PBI','i_crecimiento_PBI_capita','i_desempleo_hombres','i_desempleo_mujeres','i_Gasto_PBI','i_importaciones','migration_rate','i_desempleados_educacion','i_tasa_de_cambio','i_tasa_pobreza','i_mortalidad']

In [83]:
dataframes = {}  # Diccionario para almacenar los dataframes

for file, df_name in zip(lista_archivos, lista_nombres_df):
    dataframes[df_name] = pd.read_excel(file,sheet_name='Data',skiprows= 3)  # Leyendo el archivo y guardándolo en el diccionario

In [84]:
dataframes['i_crecimiento_PBI_capita'].head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.263941,18.866278,11.671573,2.195805,4.539317,2.370642,4.092615,4.800352,-0.567202,-1.830024,3.854744,-1.027090,-1.689537,4.918354,2.369193,-2.134006,0.117317,6.318777,-1.379148,-0.049084,1.838731,0.577088,-12.766043,-3.827013,2.400342,-1.833977,5.635318,-0.709567,2.885166,1.515938,4.917139,4.738330,0.181216,-18.698324,17.225300,NaN
1,NaN,AFE,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,-2.342974,5.093992,2.330796,1.724021,2.436215,1.09159,2.363152,1.097494,2.31429,1.721562,2.415494,-0.68631,1.492383,2.822419,-1.198882,-0.108534,-1.584489,-1.885533,-0.325151,2.362089,1.251224,-2.531377,-2.756441,0.079515,-3.301846,-0.712221,1.139694,1.062716,0.057761,-2.846680,-2.583363,-4.494923,-2.945245,-0.618975,1.664402,2.684745,1.806286,-0.655817,0.091279,0.750784,1.011096,1.262080,0.471842,2.789091,3.356423,3.754323,3.727576,1.505046,-2.035686,2.336313,0.942903,-1.844833,1.400457,1.186759,0.166840,-0.490259,-0.105886,-0.169822,-0.587557,-5.421662,1.683557,0.842507
2,Afganistán,AFG,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927029,-2.497255,7.321874,1.084988,11.803383,1.864754,17.143534,11.099220,-3.211364,8.247144,2.002522,-0.964803,-1.666416,-0.345802,-0.253594,-1.688577,0.932833,-5.364666,-22.929194,NaN


2. Creo un diccionario con los nombres de los indicadores mas cortos para que sea mas fácil leer los titulos en tablas y gráficos.

In [85]:
titulos_nuevos = [
    "Ingresos",
    "Electricidad",
    "Contaminación",
    "Crecimiento PBI",
    "Crecimiento PBI per cápita",
    "Desempleo Hombre",
    "Desempleo mujeres",
    "Gasto PBI",
    "Importaciones",
    "Migracion neta",
    "Desempleados con educación avanzada",
    "Tasa de cambio",
    "Pobreza",
    "Mortalidad"
]

titulos_viejos = ['Consumo medio o ingresos per cápita según encuestas, población total (USD por día según la PPA de 2011)',
       'Acceso a la electricidad (% de población)',
       'Contaminación del aire por PM2,5, población expuesta a niveles que exceden el valor indicativo de la Organización Mundial de la Salud (OMS) (% del total)',
       'Crecimiento del PIB (% anual)',
       'Crecimiento del PIB per cápita (% anual)',
       'Desempleo, varones (% de participación masculina en la fuerza laboral) (estimación nacional)',
       'Desempleo, mujeres (% de la población activa femenina) (estimación modelado OIT)',
       'Gasto (% del PIB)',
       'Importaciones de mercaderías (US$ a precios actuales)',
       'Migración neta',
       'Personas desempleadas con educación avanzada (% del desempleo total)',
       'Tasa de cambio oficial (UMN por US$, promedio para un período)',
       'Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)',
       'Tasa de mortalidad en un año (por cada 1.000 personas)']

diccionario_titulos = dict(zip(titulos_viejos, titulos_nuevos))

2. Creo función de transformacion que debe aplicarse a cada dataframe creado a partir de cada archivo y guardado en el diccionario dataframes. En esta funcion se pasan los años a las columas, se acortan los nombres de los indicadores y se eliminan columnas que no se necesitan

In [86]:
def transformacion (df, dic):
    
    df= df.drop(['Country Name','Indicator Code'], axis=1)
    df_melted = df.melt(id_vars=['Indicator Name','Country Code'], var_name='Year', value_name=df['Indicator Name'].max())
    df_melted=df_melted.drop(columns=['Indicator Name'])
    df_melted.rename(columns =dic, inplace=True)
    return df_melted

3. Aplico la funcion de transformacion a todos los archios y los guardo en un diccionario de dataframes df_convertidos:

In [87]:
df_convertidos={}

In [88]:
for df_name, df in dataframes.items():
    df_convertidos[df_name]=transformacion(df,diccionario_titulos)

In [89]:
df_convertidos['i_crecimiento_PBI_capita']

,Country Code,Year,Crecimiento PBI per cápita
0,ABW,1960,NaN
1,AFE,1960,NaN
2,AFG,1960,NaN
3,AFW,1960,NaN
4,AGO,1960,NaN
...,...,...,...
16753,XKX,2022,4.924753
16754,YEM,2022,NaN
16755,ZAF,2022,1.187663
16756,ZMB,2022,1.895518
